In [155]:
# run whole script
%run 03_safe_execution.ipynb

SQL blocked by guardrails: Forbidden SQL operation detected

Query: ```SELECT * FROM orders```
Valid: True
Reason: SQL is safe to execute

Query: WITH t AS (SELECT * FROM orders) SELECT * FROM t
Valid: True
Reason: SQL is safe to execute

Query: DELETE FROM orders
Valid: False
Reason: Forbidden SQL operation detected

Query: DROP TABLE products
Valid: False
Reason: Forbidden SQL operation detected

Query: SELECT * FROM orders; DELETE FROM orders
Valid: False
Reason: Forbidden SQL operation detected

Query: UPDATE orders SET order_dow = 1
Valid: False
Reason: Forbidden SQL operation detected


In [156]:
import yaml
from pathlib import Path
from typing import Tuple, Dict, Any, TypedDict, Optional, List
import json

from langgraph.graph import StateGraph, END 

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()


True

## Database Connection

In [157]:
import sys
from pathlib import Path
sys.path.append(str(Path("D:/code/text-to-sql-agent")))

from src.db.db_connection import get_db_connection

conn = get_db_connection()
cursor = conn.cursor()

## Load Schema

In [158]:
SCHEMA_PATH = Path("../src/schema/schema_summary.yaml")

with open(SCHEMA_PATH, "r", encoding="utf-8") as f:
    FULL_SCHEMA: Dict[str, Any] = yaml.safe_load(f)

print(f"✅ Loaded schema with {len(FULL_SCHEMA['tables'])} tables")


✅ Loaded schema with 6 tables


## Initialize LLm

In [159]:
from langchain_openai import ChatOpenAI
import os

def load_llm():
    return ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0,
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )

def call_llm(prompt: str) -> str:
    llm = load_llm()
    response = llm.invoke(prompt)
    return response.content.strip()


## Prompts

In [160]:
# Optimized SQL generation prompt

def build_optimized_prompt(
    question: str,
    schema: Dict[str, Any]
) -> str:
    return f"""
You are an expert PostgreSQL SQL generator.

CRITICAL RULES:
- Output ONLY one SQL SELECT query
- Do NOT include markdown, backticks, or explanations
- Use ONLY tables and columns from the schema
- Follow join templates strictly
- Never invent joins or columns
- Prefer correctness over brevity

Database schema with semantics:
{schema}

User question:
{question}

SQL:
""".strip()


# Optimized correction prompt

def build_optimized_correction_prompt(
    question: str,
    schema: Dict[str, Any],
    previous_sql: str,
    error_reason: str
) -> str:
    return f"""
You are an expert PostgreSQL SQL generator. The previous SQL query failed.

Failure reason:
{error_reason}

ABSOLUTE REQUIREMENTS (NO EXCEPTIONS):
1. You MUST output a valid SELECT query - no explanations, no refusals
2. Even if the question can't be perfectly answered, generate the CLOSEST possible SQL
3. If exact columns don't exist, use similar/related columns that DO exist
4. NEVER output text like "it is impossible" or "cannot answer"
5. If truly stuck, output: SELECT 'Data not available' as message

Available schema:
{schema}

Original question:
{question}

Failed SQL:
{previous_sql}

Now output ONLY a corrected SELECT query (no markdown, no explanations):
""".strip()

def build_validation_and_response_prompt(
    question: str,
    sql: str,
    results: list
) -> str:
    """
    Creates a prompt for LLM to:
    1. Validate if results answer the question
    2. Generate a natural language response
    """
    # Limit results to first 10 rows for efficiency
    sample_results = results[:10]
    total_rows = len(results)
    
    return f"""
You are a SQL result validator and response generator.

Your tasks:
1. Validate if the SQL query results actually answer the user's question
2. Generate a natural, conversational answer for the user

CRITICAL RULES:
- Output ONLY valid JSON with three fields: "valid" (boolean), "reason" (string), "natural_language_response" (string)
- For validation: Check if query targeted RIGHT tables/columns and results are plausible
- For response: Be conversational, clear, and directly answer the question
- Include relevant numbers/data from results
- Don't mention SQL or technical details in the natural language response
- If results are empty or wrong, explain what went wrong in a user-friendly way

User Question:
{question}

SQL Query Executed:
{sql}

Results (showing {len(sample_results)} of {total_rows} total rows):
{sample_results}

Validation Checklist:
1. Does the SQL query the correct tables/columns for this question?
2. Do the returned values semantically match what was asked?
3. Are the results plausible? (no negative counts, reasonable magnitudes)

Response Guidelines:
- Be direct and conversational
- Use numbers from the results
- Format large numbers readably (e.g., "49,688" not "49688")
- If multiple rows, summarize or show top results
- Don't say "according to the query" - just answer

Output JSON format:
{{
    "valid": true/false,
    "reason": "brief explanation of validation decision",
    "natural_language_response": "conversational answer to user's question"
}}

JSON:
""".strip()


# PLANNING NODE PROMPT

def build_planning_prompt(question: str) -> str:
    """
    Prompt for planning node to decide which tables are needed.
    """
    available_tables = list(FULL_SCHEMA['tables'].keys())
    table_descriptions = {
        name: FULL_SCHEMA['tables'][name].get('description', 'No description')
        for name in available_tables
    }
    
    return f"""
You are a database query planner. Analyze the user's question and decide which tables are needed.

Available tables:
{json.dumps(table_descriptions, indent=2)}

User question:
{question}

Your task:
1. Identify which tables are needed to answer this question
2. Consider foreign key relationships (you may need junction tables)
3. Output ONLY a JSON array of table names

Rules:
- Include ALL tables needed for joins
- If asking about products AND orders, include order_products_* tables
- Don't include unnecessary tables
- Output ONLY valid JSON, no explanation

Example outputs:
["products"]
["orders", "order_products_prior", "products"]
["products", "aisles", "departments"]

JSON array of table names:
""".strip()



## Tools

In [161]:
def schema_filter_tool(table_names: List[str]) -> Dict[str, Any]:
    """
    Extracts only specified tables and their related info from full schema.
    This is the 'tool' the agent uses to get relevant schema.
    """
    filtered_schema = {
        'tables': {},
        'hints': FULL_SCHEMA.get('hints', []),
        'common_joins': []
    }
    
    # Extract requested tables
    for table_name in table_names:
        if table_name in FULL_SCHEMA['tables']:
            filtered_schema['tables'][table_name] = FULL_SCHEMA['tables'][table_name]
    
    # Extract relevant joins (any join involving requested tables)
    for join_info in FULL_SCHEMA.get('common_joins', []):
        join_tables = set(join_info.get('tables', []))
        if join_tables.intersection(set(table_names)):
            filtered_schema['common_joins'].append(join_info)
    
    return filtered_schema


In [162]:
# # -------------------------------------------------------
# # Generate SQL (optimized)
# # -------------------------------------------------------

def generate_sql_optimized(question: str) -> str:
    prompt = build_optimized_prompt(question, FULL_SCHEMA)
    raw_sql = call_llm(prompt)
    return raw_sql


In [163]:
class SQLAgentState(TypedDict):
    question: str
    planned_tables: Optional[List[str]]  
    filtered_schema: Optional[Dict[str, Any]] 
    sql: Optional[str]
    valid: bool
    reason: Optional[str]
    retries: int
    executed: bool          
    results: Optional[list]
    nl_response: Optional[str]

    failure_type: Optional[str]  
    attempted_strategies: List[str]  
    current_strategy: str  

In [164]:
def planning_node(state: SQLAgentState) -> SQLAgentState:
    """
    Analyzes question and decides which tables are needed.
    This is what makes it AGENTIC - the agent plans before acting.
    """
    print("🧠 Planning: Analyzing question...")
    
    prompt = build_planning_prompt(state["question"])
    response = call_llm(prompt)
    
    try:
        # Parse the JSON response
        planned_tables = json.loads(response)
        
        if not isinstance(planned_tables, list):
            print("⚠️ Planning failed: Invalid response format")
            # Fallback: use all tables
            planned_tables = list(FULL_SCHEMA['tables'].keys())
        
        print(f"📋 Plan: Need tables {planned_tables}")
        
        # Use schema filter tool to get relevant schema
        filtered_schema = schema_filter_tool(planned_tables)
        print(f"✂️ Filtered schema: {len(filtered_schema['tables'])} tables, {len(filtered_schema['common_joins'])} joins")
        
        return {
            **state, 
            "planned_tables": planned_tables,
            "filtered_schema": filtered_schema
        }
    
    except json.JSONDecodeError as e:
        print(f"⚠️ Planning failed to parse JSON: {e}")
        print(f"Raw response: {response[:200]}")
        # Fallback: use all tables
        return {
            **state,
            "planned_tables": list(FULL_SCHEMA['tables'].keys()),
            "filtered_schema": FULL_SCHEMA
        }



In [165]:

def generate_sql_node(state: SQLAgentState) -> SQLAgentState:
    """
    NOW uses filtered_schema instead of full schema.
    This is the benefit of planning!
    """
    print("🔄 Generating SQL...")
    
    # Use filtered schema from planning node
    schema_to_use = state.get("filtered_schema", FULL_SCHEMA)
    
    prompt = build_optimized_prompt(state["question"], schema_to_use)
    raw_sql = call_llm(prompt)
    sql = clean_sql(raw_sql)
    
    print(f"Generated: {sql[:100]}...")
    return {**state, "sql": sql}


In [166]:
def validate_sql_node(state: SQLAgentState) -> SQLAgentState:
    print("🔍 Validating syntax...")
    is_valid, reason = validate_sql(state["sql"])
    print(f"Syntax valid: {is_valid}")
    if not is_valid:
        print(f"Reason: {reason}")
    return {**state, "valid": is_valid, "reason": None if is_valid else reason, "executed": False}

In [167]:
def execute_sql_node(state: SQLAgentState) -> SQLAgentState:
    """Execute SQL with proper transaction management."""
    print("⚡ Executing SQL...")
    try:
        cursor.execute(state["sql"])
        results = cursor.fetchall()
        conn.commit()  # ADD THIS: Commit successful query
        print(f"✅ Executed! Got {len(results)} rows")
        return {**state, "executed": True, "results": results, "reason": None}
    except Exception as e:
        conn.rollback()  # ⭐ ADD THIS: Critical fix!
        print(f"❌ Execution failed: {str(e)[:100]}")
        print("🔄 Transaction rolled back")  # ADD THIS
        return {**state, "executed": False, "results": None, "reason": f"Execution error: {str(e)}"}

In [168]:
def validate_execution_node(state: SQLAgentState) -> SQLAgentState:
    print("🔍 Validating execution...")
    if not state["executed"]:
        print("Failed - execution error")
        return {**state, "valid": False}
    if not state["results"] or len(state["results"]) == 0:
        print("Failed - no results")
        return {**state, "valid": False, "reason": "Query returned no results"}
    print("✅ Validation passed!")
    return {**state, "valid": True, "reason": None}

In [169]:
MAX_RETRIES = 2 

def correct_sql_node(state: SQLAgentState) -> SQLAgentState:
    print(f"🔧 Correcting SQL (retry {state['retries'] + 1}/{MAX_RETRIES})...")
    
    schema_to_use = state.get("filtered_schema", FULL_SCHEMA)
    
    prompt = build_optimized_correction_prompt(
        question=state["question"],
        schema=schema_to_use,
        previous_sql=state["sql"],
        error_reason=state["reason"]
    )
    corrected_sql = call_llm(prompt)
    sql = clean_sql(corrected_sql)
    print(f"Corrected: {sql[:100]}...")
    
    # ⭐⭐⭐ ADD THESE LINES ⭐⭐⭐
    attempted = state.get("attempted_strategies", [])
    if "correct" not in attempted:
        attempted.append("correct")
    
    return {
        **state, 
        "sql": sql, 
        "retries": state["retries"] + 1,
        "attempted_strategies": attempted  # ⭐⭐⭐ ADD THIS ⭐⭐⭐
    }

In [170]:
def validate_answer_and_generate_response_node(state: SQLAgentState) -> SQLAgentState:
    """
    Validates if SQL results answer the question AND generates natural language response.
    Does both in a single LLM call for efficiency.
    """
    print("🔍 Validating answer + generating response...")
    
    if not state["executed"] or not state["results"]:
        print("⚠️ Cannot validate - no results to check")
        return {
            **state, 
            "valid": False, 
            "reason": "No results to validate",
            "nl_response": "I couldn't execute the query to get an answer."
        }
    
    # Build combined validation + NL generation prompt
    prompt = build_validation_and_response_prompt(
        question=state["question"],
        sql=state["sql"],
        results=state["results"]
    )
    
    # Get LLM judgment
    response = call_llm(prompt)
    
    # Parse response (expecting JSON format)
    try:
        import json
        output = json.loads(response)
        
        is_valid = output.get("valid", False)
        reason = output.get("reason", "Unknown validation failure")
        nl_response = output.get("natural_language_response", "Unable to generate response.")
        
        if is_valid:
            print("✅ Answer validated! Generated NL response.")
            print(f"📝 Response: {nl_response[:100]}...")
        else:
            print(f"❌ Answer validation failed: {reason}")
        
        return {
            **state, 
            "valid": is_valid, 
            "reason": None if is_valid else reason,
            "nl_response": nl_response
        }
    
    except json.JSONDecodeError as e:
        print(f"⚠️ Failed to parse validation response: {e}")
        return {
            **state, 
            "valid": False, 
            "reason": "Validation parsing error",
            "nl_response": "Error processing the query results."
        }

In [171]:
def analyze_failure_node(state: SQLAgentState) -> SQLAgentState:
    """
    AGENTIC: Agent analyzes WHY it failed and decides next strategy.
    This is what makes it truly agentic - intelligent decision making.
    """
    print("🧠 Analyzing failure...")
    
    reason = state.get("reason", "")
    
    # Agent classifies failure type
    if "syntax" in reason.lower() or "invalid" in reason.lower():
        failure_type = "syntax_error"
        print("📊 Failure type: Syntax error")
    elif "no results" in reason.lower() or "empty" in reason.lower():
        failure_type = "no_results"
        print("📊 Failure type: No results returned")
    elif "execution" in reason.lower():
        failure_type = "execution_error"
        print("📊 Failure type: Execution error")
    elif not state.get("valid", False) and state.get("executed", False):
        failure_type = "semantic_error"
        print("📊 Failure type: Answer doesn't match question")
    else:
        failure_type = "unknown"
        print("📊 Failure type: Unknown")
    
    return {**state, "failure_type": failure_type}

In [172]:
def generate_simplified_sql_node(state: SQLAgentState) -> SQLAgentState:
    """
    STRATEGY B: Try a simpler query approach.
    Agent decides to use this when complexity is the issue.
    """
    print("🔄 Strategy: Generating SIMPLIFIED SQL...")
    
    prompt = f"""
You are an expert PostgreSQL SQL generator.

The previous complex query failed. Generate a SIMPLER query.

CRITICAL RULES:
- Use the SIMPLEST approach possible
- Avoid complex joins if possible
- Use single table if feasible
- Output ONLY SQL, no markdown

Database schema:
{FULL_SCHEMA}

User question:
{state['question']}

Previous failed SQL (too complex):
{state['sql']}

Failure reason:
{state['reason']}

Generate SIMPLER SQL:
""".strip()
    
    raw_sql = call_llm(prompt)
    sql = clean_sql(raw_sql)
    print(f"Simplified: {sql[:100]}...")
    
    attempted = state.get("attempted_strategies", [])
    attempted.append("simplified")
    
    return {
        **state, 
        "sql": sql, 
        "current_strategy": "simplified",
        "attempted_strategies": attempted,
        "retries": 0
    }


def generate_alternative_approach_node(state: SQLAgentState) -> SQLAgentState:
    """
    STRATEGY C: Try a completely different approach.
    Agent decides to rethink the problem from scratch.
    """
    print("🔄 Strategy: Trying ALTERNATIVE approach...")
    
    prompt = f"""
You are an expert PostgreSQL SQL generator.

Previous attempts failed. Think differently about this problem.

CRITICAL RULES:
- Approach this from a DIFFERENT angle
- Consider alternative tables or join patterns
- Use different aggregation methods if applicable
- Output ONLY SQL, no markdown

Database schema:
{FULL_SCHEMA}

User question:
{state['question']}

Previous attempts that failed:
{state.get('attempted_strategies', [])}

Previous SQL:
{state['sql']}

Failure reason:
{state['reason']}

Generate ALTERNATIVE SQL approach:
""".strip()
    
    raw_sql = call_llm(prompt)
    sql = clean_sql(raw_sql)
    print(f"Alternative: {sql[:100]}...")
    
    attempted = state.get("attempted_strategies", [])
    attempted.append("alternative")
    
    return {
        **state, 
        "sql": sql, 
        "current_strategy": "alternative",
        "attempted_strategies": attempted,
        "retries": 0
    }


def ask_clarification_node(state: SQLAgentState) -> SQLAgentState:
    """
    FINAL FALLBACK: Agent admits it needs help and asks user.
    This shows intelligence - knowing when to ask for help.
    """
    print("💬 Strategy: Asking user for clarification...")
    
    clarification = f"""I tried multiple approaches but couldn't answer your question: "{state['question']}"

Attempts made:
{', '.join(state.get('attempted_strategies', ['direct']))}

Last error: {state['reason']}

Could you please:
- Rephrase your question, or
- Provide more specific details, or
- Break it into smaller questions?"""
    
    return {
        **state,
        "nl_response": clarification,
        "valid": False
    }

In [173]:
MAX_RETRIES = 2

def route_after_syntax_check(state: SQLAgentState):
    """Route after pre-execution validation"""
    if state["valid"]:
        return "execute_sql"
    if state["retries"] >= MAX_RETRIES:
        return END
    return "correct_sql"

def route_after_execution(state: SQLAgentState):
    """Route after execution - goes to combined validation+response node"""
    if state["executed"]:
        return "validate_and_respond"
    if state["retries"] >= MAX_RETRIES:
        return END
    return "correct_sql"

def route_after_validation(state: SQLAgentState):
    """
    AGENTIC ROUTING: Agent decides next strategy based on failure analysis.
    This is TRUE agency - choosing different paths based on context.
    """
    if state["valid"]:
        return END  # Success!
    
    # Analyze failure first
    return "analyze_failure"

def route_after_failure_analysis(state: SQLAgentState):
    """
    AGENTIC: Agent chooses strategy based on failure type and attempts.
    """
    attempted = state.get("attempted_strategies", [])
    failure_type = state.get("failure_type", "unknown")
    retries = state.get("retries", 0)
    
    print(f"🤔 Agent deciding next move...")
    print(f"   Attempts so far: {attempted}")
    print(f"   Failure type: {failure_type}")
    print(f"   Retries: {retries}/{MAX_RETRIES}")
    
    # If we've exhausted retries, escalate strategies
    if retries >= MAX_RETRIES:
        print("   ⚠️ Max retries reached, escalating strategy")
        
        # Escalation ladder based on what we've tried
        if "simplified" not in attempted and len(attempted) < 3:
            print("   Decision: Try simplified SQL approach")
            return "generate_simplified"
        
        elif "alternative" not in attempted and len(attempted) < 4:
            print("   Decision: Try completely different approach")
            return "generate_alternative"
        
        else:
            print("   Decision: Ask user for clarification (all strategies exhausted)")
            return "ask_clarification"
    
    # First few attempts - try simple correction
    if "correct" not in attempted or len(attempted) == 0:
        print("   Decision: Try correcting the SQL")
        return "correct_sql"
    
    # If correction failed multiple times, try different strategy
    if failure_type in ["syntax_error", "semantic_error"]:
        if "simplified" not in attempted:
            print("   Decision: Try simplified SQL approach")
            return "generate_simplified"
        elif "alternative" not in attempted:
            print("   Decision: Try alternative approach")
            return "generate_alternative"
    
    # Default: one more correction attempt
    print("   Decision: Try correcting again")
    return "correct_sql"

In [174]:
graph = StateGraph(SQLAgentState)

# Nodes
graph.add_node("planning", planning_node)  # NEW: Entry point!
graph.add_node("generate_sql", generate_sql_node)
graph.add_node("validate_sql", validate_sql_node)
graph.add_node("execute_sql", execute_sql_node)
graph.add_node("validate_and_respond", validate_answer_and_generate_response_node)
graph.add_node("correct_sql", correct_sql_node)

graph.add_node("analyze_failure", analyze_failure_node)
graph.add_node("generate_simplified", generate_simplified_sql_node)
graph.add_node("generate_alternative", generate_alternative_approach_node)
graph.add_node("ask_clarification", ask_clarification_node)

# NEW: Entry point is now planning
graph.set_entry_point("planning")

# Flow
graph.add_edge("planning", "generate_sql")  # NEW: Planning → Generation
graph.add_edge("generate_sql", "validate_sql")

graph.add_conditional_edges(
    "validate_sql",
    route_after_syntax_check,
    {
        "execute_sql": "execute_sql",
        "correct_sql": "correct_sql",
        END: END
    }
)

graph.add_conditional_edges(
    "execute_sql",
    route_after_execution,
    {
        "validate_and_respond": "validate_and_respond",
        "correct_sql": "correct_sql",
        END: END
    }
)

# NEW: Route to failure analysis first
graph.add_conditional_edges(
    "validate_and_respond",
    route_after_validation,
    {
        "analyze_failure": "analyze_failure",
        END: END
    }
)

# NEW: Agent decides strategy after analyzing failure
graph.add_conditional_edges(
    "analyze_failure",
    route_after_failure_analysis,
    {
        "correct_sql": "correct_sql",
        "generate_simplified": "generate_simplified",
        "generate_alternative": "generate_alternative",
        "ask_clarification": "ask_clarification"
    }
)

# NEW: All strategies go back to validation
graph.add_edge("generate_simplified", "validate_sql")
graph.add_edge("generate_alternative", "validate_sql")
graph.add_edge("ask_clarification", END)  

graph.add_edge("correct_sql", "validate_sql")

# Compile
sql_agent = graph.compile()

In [175]:
initial_state: SQLAgentState = {
    # "question": "Show me the top 3 most ordered products",
    "question": "What's the average profit margin per product category last month?",
    "sql": None,
    "valid": False,
    "reason": None,
    "retries": 0,
    "executed": False,
    "results": None,
    "nl_response": None,
    # NEW
    "failure_type": None,
    "attempted_strategies": [],
    "current_strategy": "direct"
}

final_state = sql_agent.invoke(initial_state)

# Enhanced output
print("\n" + "="*60)
print("🤖 AGENTIC SQL AGENT RESULTS")
print("="*60)

if final_state["valid"] and final_state["executed"]:
    print("\n✅ SUCCESS")
    print(f"\n🙋 Question: {final_state['question']}")
    print(f"\n🧠 Planned Tables: {final_state['planned_tables']}")
    print(f"\n💬 Answer: {final_state['nl_response']}")
    print(f"\n🔧 SQL: {final_state['sql']}")
    print(f"\n📊 Raw Results: {final_state['results'][:3]}")
else:
    print("\n❌ FAILED")
    print(f"\n🙋 Question: {final_state['question']}")
    print(f"\n🧠 Planned Tables: {final_state.get('planned_tables', 'Not planned')}")
    print(f"\n⚠️ Error: {final_state['reason']}")
    print(f"\n💬 Response: {final_state['nl_response']}")
    print(f"\n🔄 Retries: {final_state['retries']}/{MAX_RETRIES}")

print("="*60)

🧠 Planning: Analyzing question...
📋 Plan: Need tables ['orders', 'order_products_prior', 'products', 'departments']
✂️ Filtered schema: 4 tables, 5 joins
🔄 Generating SQL...
Generated: select d.department, avg(pm.profit_margin) as average_profit_margin from products p inner join depar...
🔍 Validating syntax...
Syntax valid: True
⚡ Executing SQL...
❌ Execution failed: relation "product_margins" does not exist
LINE 1: ...d on p.department_id = d.department_id inner jo
🔄 Transaction rolled back
🔧 Correcting SQL (retry 1/2)...
Corrected: select d.department, avg(0) as average_profit_margin from products p inner join departments d on p.d...
🔍 Validating syntax...
Syntax valid: True
⚡ Executing SQL...
✅ Executed! Got 21 rows
🔍 Validating answer + generating response...
❌ Answer validation failed: The query calculates the average profit margin as zero for all product categories by using a fixed value of 0 instead of actual profit margin data. It also does not filter for last month's data, so 

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT